In [572]:
import toy_data.sine_wave as swg
import numpy as np
import bokeh.io
import bokeh.plotting as bp
import tensorflow as tf
bokeh.io.output_notebook()

d_x = np.array([int(i) for i in '0010010010010010010'])[np.newaxis, np.newaxis, :]
d_X = np.concatenate([d_x]*10, axis=0)

Loading BokehJS ...

In [573]:
d_X.shape

(10, 1, 19)

In [1619]:
tf.reset_default_graph()
dim_s = 2
dim_x = 1
get_x = lambda s: s[:, 0:dim_x]
x_len = d_X.shape[2]

io_tf = lambda dim: tf.placeholder(tf.float32, dim)

x_in = io_tf([None, dim_x, x_len])
s0 = tf.Variable(tf.random_uniform([1, dim_s]), name="s_0")
T = tf.Variable(tf.random_uniform([dim_s, dim_s]), name="Transition")
T0 = tf.Variable(tf.random_uniform([1, dim_s]), name="Transition")
s = tf.sigmoid(tf.matmul(s0, T) + T0)


x_out_0 = get_x(s)
xt = x_out_0
x_in_0 = x_in[:,:,0]

x_logits = get_x(tf.matmul(s0, T) + T0)
cost_is  = tf.map_fn(lambda _l: tf.nn.sigmoid_cross_entropy_with_logits(x_logits, _l), x_in_0)
cost = tf.reduce_sum(cost_is)
# cost = -(tf.reduce_sum(x_in_0*tf.log(x_out_0) + (1 - x_in_0)*tf.log(1 - x_out_0), axis=0))
cost_ts = [cost]
x_in_ts = x_in_0[0]

for t in range(1, x_len):
    x_in_t = x_in[:,:,t]
    x_in_ts = tf.concat(0, (x_in_ts, x_in_t[0]))
    
    x_logits = get_x(tf.matmul(s, T) + T0)
    cost_is  = tf.map_fn(lambda _l: tf.nn.sigmoid_cross_entropy_with_logits(x_logits, _l), x_in_t)
    cost_t = tf.reduce_sum(cost_is)
    
    s = tf.sigmoid(tf.matmul(s, T) + T0)
    x_out_t = get_x(s)
    xt = tf.concat(0, (xt, x_out_t))
    # cost_t = -(tf.reduce_sum(x_in_t*tf.log(x_out_t) + (1 - x_in_t)*tf.log(1 - x_out_t), axis=0))
    cost += cost_t
    cost_ts.append(cost_t)

train_step = tf.train.RMSPropOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [1634]:
for i in range(100):
    sess.run(train_step, feed_dict={x_in: d_X})

print(sess.run(cost, feed_dict={x_in: d_X}))
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
print(np.array(sess.run(cost_ts, feed_dict={x_in: d_X})))
print(sess.run(x_in_ts, feed_dict={x_in: d_X}))
print(sess.run(xt, feed_dict={x_in: d_X}))
print(sess.run(s0, feed_dict={x_in: d_X}))
w = sess.run(T, feed_dict={x_in: d_X})
p = bp.figure(plot_width=300, plot_height=300, 
              x_range=[0, w.shape[0]], y_range=[0, w.shape[1]])
p.image(image=[np.flipud(w)], x=0, y=0, dw=w.shape[0], dh=w.shape[1], palette="Viridis11")
print(w)
bp.show(p)
# print(d_x[:])

0.152498
[ 0.00  0.01  0.01  0.00  0.01  0.01  0.00  0.01  0.01  0.00  0.01  0.01
  0.00  0.01  0.01  0.00  0.01  0.01  0.00]
[ 0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00
  0.00  0.00  1.00  0.00  0.00  1.00  0.00]
[ 0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00
  0.00  0.00  1.00  0.00  0.00  1.00  0.00]
[[ 3.82  1.03]]
[[-11.86  13.63]
 [-13.36 -9.40]]
